# Домашнее задание 3. Fine-Tuning модели BERT и анализ альтернативных архитектур в задаче классификации

**ФИО Студента: Кузнецов Кирилл Игоревич**

**Дата Выполнения:**

---

### **Описание задания**

В этом задании вы реализуете эксперементальное сравнение классического трансформера (BERT) с современными альтернативными архитектурами (Mamba) на задаче классификации русскоязычных текстов. Проведете исследование trade-offs между качеством, скоростью и количеством обучаемых параметров для различных подходов к Fine-Tuning.
---

## **Установка и импорт библиотек**

In [6]:
# # Установка PEFT для LoRA
# !pip install peft

# # Установка зависимостей для Mamba
# !pip install causal-conv1d mamba-ssm


In [5]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import time
import warnings

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed
)

from datasets import Dataset as HFDataset
import evaluate

from peft import (
    LoraConfig,
    TaskType,
    get_peft_model
)

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

---
## **Задание 1. Подготовка данных и базовой модели**

Используем полный датасет русскоязычных отзывов с Кинопоиска. Для упрощения задачи бинарной классификации удаляем нейтральные отзывы. Разбиваем данные на обучающую и тестовую выборки в соотношении 80/20.

Задачи:
1. Загрузите датасет отзывов Кинопоиска и соответствующий токенизатор для DeepPavlov/rubert-base-cased.


In [ ]:
# Загружаем полный датасет
print("Загружаем полный датасет отзывов...")
df_full = pd.read_json("hf://datasets/blinoff/kinopoisk/kinopoisk.jsonl", lines=True)

# ...

Определим функции для токенизации, вычисления метрик и подсчета обучаемых параметров, которые будут использоваться для всех моделей.   

2. Подготовьте данные: создайте dataset-объекты для обучающей и тестовой выборок, токенизируйте тексты и подготовьте их к подаче в модель в соответствии с семинаром 1 данной дисциплины.  
3. Определите функцию для вычисления метрик Accuracy, F1-score.  




  

In [ ]:
MAX_LENGTH = 256 # Ограничиваем длину для ускорения обучения и экономии памяти

# ...

---
## **Задания 2 и 3. Baseline — Fine-Tuning BERT**

В качестве baseline используем русскоязычную модель `DeepPavlov/rubert-base-cased`. Мы рассмотрим два подхода: полный Fine-Tuning и эффективный Fine-Tuning с помощью LoRA.

Задачи:  
**BERT Full Fine-Tuning:**
1. Загрузите предобученную модель DeepPavlov/rubert-base-cased.
2. Настройте TrainingArguments для полного дообучения.
3. Обучите модель на полном обучающем наборе данных.
4. Оцените качество на тестовой выборке и зафиксируйте время обучения и количество обучаемых параметров.  

**BERT с LoRA или иным методом (Parameter-Efficient Fine-Tuning):**
1. Снова загрузите исходную модель DeepPavlov/rubert-base-cased.
2. Настройте LoraConfig, указав целевые модули (например, query, value).
3. Примените LoRA к модели с помощью get_peft_model.
4. Обучите параметро-эффективную модель.
5. Оцените ее качество, время обучения и количество обучаемых параметров.   
6. Сравните с результатами полного дообучения.

In [ ]:
MODEL_NAME_BERT = "DeepPavlov/rubert-base-cased"

In [ ]:
print("--- 1. BERT: Full Fine-tuning ---")

# ...

--- 1. BERT: Full Fine-tuning ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.154200,0.153269,0.947837,0.946175



Результаты для BERT Full Fine-tuning:
{'Accuracy': 0.9478369514290177, 'F1-Score': 0.9461745148484598, 'Trainable Params': 177854978, 'Training Time (s)': 1374.4948289394379}


In [ ]:
print("\n--- 2. BERT: LoRA Fine-tuning ---")

# ...

---
## **Задание 4. Альтернативная архитектура — Mamba**

Mamba — это современная архитектура на основе пространств состояний (State Space Models), которая показывает высокую производительность и линейную сложность по длине последовательности. Используем небольшую предобученную модель `state-spaces/mamba-130m-hf`.

Архитектура еще не распространенная, поэтому нужно самостоятельно написать блок для классификатора последовательностей. Вы можете воспользоваться готовым кодом ниже для эксперимента, либо установить библиотеку https://github.com/getorca/mamba_for_sequence_classification, либо протестировать иные архитектуры с huggingface

Задачи:  
1. Загрузите предобученную модель Mamba или другую, подходящую для классификации текста (например, state-spaces/mamba-130m-hf).
2. Адаптируйте модель для задачи бинарной классификации (добавьте классификационную голову).
3. Настройте TrainingArguments и проведите Fine-Tuning модели Mamba.
Оцените ее итоговое качество, время обучения и количество параметров.


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput

# --- 1. Создаем наш собственный класс для классификации ---
class CustomMambaForSequenceClassification(nn.Module):
    def __init__(self, model_name="state-spaces/mamba-130m-hf", num_labels=2):
        super().__init__()
        self.num_labels = num_labels

        # Загружаем базовую модель Mamba (без головы для конкретной задачи)
        self.mamba = AutoModel.from_pretrained(model_name)

        # Получаем размер скрытого состояния из конфигурации модели
        hidden_size = self.mamba.config.hidden_size

        # Создаем голову для классификации — обычный линейный слой
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Прогоняем данные через базовую модель Mamba
        outputs = self.mamba(input_ids=input_ids, attention_mask=attention_mask)

        # Mamba возвращает last_hidden_state. Его форма: (batch_size, sequence_length, hidden_size)
        last_hidden_state = outputs.last_hidden_state

        # Для классификации берем скрытое состояние ПОСЛЕДНЕГО токена в последовательности
        # Это стандартная практика для авторегрессионных моделей, как Mamba
        cls_embedding = last_hidden_state[:, -1, :]

        # Прогоняем его через наш классификатор, чтобы получить логиты
        logits = self.classifier(cls_embedding)

        # Если переданы метки (labels), вычисляем loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Возвращаем результат в формате, который понимает Trainer
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=None, # Mamba не использует Attention, поэтому None
        )


MODEL_NAME_MAMBA = "state-spaces/mamba-130m-hf"
TOKENIZER_NAME_MAMBA = "EleutherAI/gpt-neox-20b"

# Загружаем токенизатор
tokenizer_mamba = AutoTokenizer.from_pretrained(TOKENIZER_NAME_MAMBA)
if tokenizer_mamba.pad_token is None:
    tokenizer_mamba.pad_token = tokenizer_mamba.eos_token
print("Токенизатор для Mamba успешно загружен.")

# Токенизируем датасеты
tokenized_train_mamba, tokenized_test_mamba = create_tokenized_datasets(tokenizer_mamba, train_dataset, test_dataset)

# --- 2. Инициализируем нашу кастомную модель ---
model_mamba = CustomMambaForSequenceClassification(model_name=MODEL_NAME_MAMBA, num_labels=2)
model_mamba.mamba.config.pad_token_id = tokenizer_mamba.pad_token_id


Токенизатор для Mamba успешно загружен.


Map:   0%|          | 0/25612 [00:00<?, ? examples/s]

Map:   0%|          | 0/6403 [00:00<?, ? examples/s]

In [ ]:
# --- 3. Обучаем как обычно с помощью Trainer ---
training_args_mamba = TrainingArguments(
    output_dir="./results/mamba_custom",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_dir='./logs/mamba_custom',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

trainer_mamba = Trainer(
    model=model_mamba,
    args=training_args_mamba,
    train_dataset=tokenized_train_mamba,
    eval_dataset=tokenized_test_mamba,
    tokenizer=tokenizer_mamba,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer_mamba)
)

start_time = time.time()
trainer_mamba.train()
train_time = time.time() - start_time

eval_results = trainer_mamba.evaluate()
results['Mamba'] = {
    'Accuracy': eval_results['eval_accuracy'],
    'F1-Score': eval_results['eval_f1'],
    'Trainable Params': count_trainable_parameters(model_mamba),
    'Training Time (s)': train_time
}

print(f"\nРезультаты для Mamba Fine-tuning (кастомная модель):")
print(results['Mamba'])

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.375400,0.329374,0.915352,0.914519



Результаты для Mamba Fine-tuning (кастомная модель):
{'Accuracy': 0.9153521786662502, 'F1-Score': 0.9145188787765379, 'Trainable Params': 129136898, 'Training Time (s)': 1825.4752488136292}


---
## **Задание 5. Сравнительный анализ и выводы**

Проведите сравнительный анализ подходов и сделайте выводы на основе проведенных эксперементов.

Задачи:  
1. Создайте сводную таблицу, в которой будут отражены все ключевые показатели для трех подходов:
- BERT Full Fine-Tuning
- BERT + LoRA
- Mamba
2. Сравните модели по следующим критериям:
- Качество: Accuracy и F1-score.
- Эффективность: время обучения и количество обучаемых параметров.

3. Сформулируйте развернутые выводы:
- Какой подход показал наилучшее качество?
- Насколько LoRA сокращает количество параметров и время обучения по сравнению с полным Fine-Tuning? Как это влияет на метрики?
- Как Mamba показывает себя в сравнении с BERT? В чем ее сильные и слабые стороны для данной задачи?

4. Дайте рекомендации по выбору архитектуры в зависимости от ограничений (время, вычислительные ресурсы, требования к качеству).


In [ ]:
# ...